<a href="https://colab.research.google.com/github/Emmanuel-K07/chatterbox/blob/master/stable/526_mix_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Étape 1 : Purge complète pour repartir propre
!rm -rf /usr/local/lib/python3.11/dist-packages/numpy* \
       /usr/local/lib/python3.11/dist-packages/~umpy* \
       /usr/local/lib/python3.11/dist-packages/PIL \
       /usr/local/lib/python3.11/dist-packages/torch* \
       /usr/local/lib/python3.11/dist-packages/torchvision* \
       /usr/local/lib/python3.11/dist-packages/torchaudio* \
       /usr/local/lib/python3.11/dist-packages/xformers* \
       /usr/local/lib/python3.11/dist-packages/pillow*

!pip cache purge -q

# Étape 2 : Installation cohérente (pas de versions qui se remplacent)
!pip install --no-cache-dir torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install --no-cache-dir xformers==0.0.20 triton==2.0.0
!pip install --no-cache-dir numpy==1.26.4 mediapipe==0.10.15
!pip install --no-cache-dir pillow==10.2.0 addict yapf fvcore omegaconf

# Étape 3 : Patch du bug PIL.Image.LINEAR (sans downgrade Pillow)
import fileinput, sys
import os

transform_py = "/content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors/v11/oneformer/detectron2/data/transforms/transform.py"

if os.path.exists(transform_py):
    with fileinput.FileInput(transform_py, inplace=True) as file:
        for line in file:
            # Remplace Image.LINEAR par Image.Resampling.BILINEAR (Pillow >=10)
            sys.stdout.write(line.replace("Image.LINEAR", "Image.Resampling.BILINEAR"))

# Étape 4 : Lancer ComfyUI
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared && chmod +x /content/cloudflared

import atexit, subprocess, time, re, requests
from random import randint
from threading import Timer
from queue import Queue

def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(
        ['/content/cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'],
        stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)",
                                   requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Impossible de connecter à Cloudflare")
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
print(f"Accès WebUI: {output_queue.get()}")

!python /content/ComfyUI/main.py --dont-print-server


In [ ]:
pip install --force-reinstall torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio --index-url https://download.pytorch.org/whl/cu118
